In [1]:
import numpy as np

def trimf(x, a, b, c):
    """Fungsi keanggotaan Segitiga (Triangular Membership Function)"""
    return max(min((x - a) / (b - a), (c - x) / (c - b)), 0) if b != a and c != b else 0

def trapmf_left(x, b, c):
    """Trapesium kiri (turun)"""
    if x <= b: return 1
    if x >= c: return 0
    return (c - x) / (c - b)

def trapmf_right(x, a, b):
    """Trapesium kanan (naik)"""
    if x <= a: return 0
    if x >= b: return 1
    return (x - a) / (b - a)

# ==========================================
# BAGIAN A: FUZZY LOGIC (KASUS DBD)
# Metode: Tsukamoto (Monotonik Output)
# ==========================================
print("--- HASIL BAGIAN A: DIAGNOSA DBD (TSUKAMOTO) ---")

# 1. Input Data [cite: 6]
suhu = 38.5
trombosit = 120
leukosit = 3.8

# 2. Fuzzifikasi Input [cite: 3, 4, 5]
# Suhu
mu_suhu_normal = 0 # 38.5 diluar 36-37.5
mu_suhu_demam  = trimf(suhu, 37, 38, 39) # Asumsi puncak di 38
mu_suhu_tinggi = trapmf_right(suhu, 38, 41)

# Trombosit (<150 Rendah)
mu_trom_rendah = trapmf_left(trombosit, 150, 150) # Strict cutoff based on logic
mu_trom_normal = trapmf_right(trombosit, 150, 400)

# Leukosit (<4 Rendah)
mu_leuk_rendah = trapmf_left(leukosit, 4, 4)
mu_leuk_normal = trapmf_right(leukosit, 4, 10)

print(f"Fuzzifikasi Suhu ({suhu}): Demam={mu_suhu_demam:.3f}, Tinggi={mu_suhu_tinggi:.3f}")

# 3. Inferensi (Rule Evaluation) & Kalkulasi Z (Output) [cite: 6]
# Output 'Tinggi' Range 60-100. Fungsi invers: z = alpha * (100-60) + 60
z_results = []
alphas = []

# Rule 1: IF Suhu Tinggi AND Trombosit Rendah AND Leukosit Rendah THEN Risiko Tinggi
a1 = min(mu_suhu_tinggi, mu_trom_rendah, mu_leuk_rendah)
z1 = (a1 * 40) + 60 # Invers fungsi naik (60 ke 100)
alphas.append(a1); z_results.append(z1)

# Rule 2: IF Suhu Demam AND Trombosit Rendah THEN Risiko Tinggi
a2 = min(mu_suhu_demam, mu_trom_rendah)
z2 = (a2 * 40) + 60
alphas.append(a2); z_results.append(z2)

# Rule 3, 4, 5 (Lainnya akan bernilai 0 atau tidak aktif berdasarkan input)
# Kita skip untuk efisiensi karena alpha pasti 0 jika komponen input 0.

# 4. Defuzzifikasi (Weighted Average)
numerator = sum(a * z for a, z in zip(alphas, z_results))
denominator = sum(alphas)

if denominator != 0:
    risiko_dbd = numerator / denominator
else:
    risiko_dbd = 0

print(f"Alpha Rule 1: {a1:.3f}, Z1: {z1:.3f}")
print(f"Alpha Rule 2: {a2:.3f}, Z2: {z2:.3f}")
print(f"Diagnosa Risiko DBD: {risiko_dbd:.4f}")
print("\n" + "="*40 + "\n")


# ==========================================
# BAGIAN B: ANFIS (KASUS JAGUNG)
# Metode: Sugeno Orde Satu
# ==========================================
print("--- HASIL BAGIAN B: PREDIKSI JAGUNG (ANFIS SUGENO) ---")

# 1. Input Data
x1 = 60 # Soil Moisture
x2 = 26 # Avg Temp

# 2. Fuzzifikasi Input [cite: 7, 8]
# Soil Moisture (0, 30, 60) & (40, 70, 100)
mu_x1_low  = trimf(x1, 0, 30, 60)
mu_x1_high = trimf(x1, 40, 70, 100)

# Avg Temp (15, 20, 25) & (22, 28, 35)
mu_x2_low  = trimf(x2, 15, 20, 25)
mu_x2_high = trimf(x2, 22, 28, 35)

print(f"Fuzzifikasi Moisture ({x1}): Low={mu_x1_low}, High={mu_x1_high:.3f}")
print(f"Fuzzifikasi Temp ({x2}): Low={mu_x2_low}, High={mu_x2_high:.3f}")

# 3. Firing Strength (Bobot w) - Menggunakan PRODUCT (Standard ANFIS)
w1 = mu_x1_low * mu_x2_low
w2 = mu_x1_low * mu_x2_high
w3 = mu_x1_high * mu_x2_low
w4 = mu_x1_high * mu_x2_high

total_w = w1 + w2 + w3 + w4
print(f"Weights: w1={w1}, w2={w2}, w3={w3}, w4={w4:.4f}")

# 4. Normalisasi Bobot (w_bar)
w1_bar = w1 / total_w if total_w != 0 else 0
w2_bar = w2 / total_w if total_w != 0 else 0
w3_bar = w3 / total_w if total_w != 0 else 0
w4_bar = w4 / total_w if total_w != 0 else 0

# 5. Konsekuen Output (y = px1 + qx2 + r)
# Parameter [p, q, r]
y1 = (0.0692 * x1) + (0.0461 * x2) + 0.00231
y2 = (0.0698 * x1) + (0.0390 * x2) + 0.00139
y3 = (0.0585 * x1) + (0.0183 * x2) + 0.00084
y4 = (0.0505 * x1) + (0.0318 * x2) + 0.00115

print(f"Output Rule 4 (y4): {y4:.5f}")

# 6. Output Akhir (Weighted Sum)
output_anfis = (w1_bar * y1) + (w2_bar * y2) + (w3_bar * y3) + (w4_bar * y4)

print(f"Prediksi Produksi Jagung: {output_anfis:.4f} ton/ha")

--- HASIL BAGIAN A: DIAGNOSA DBD (TSUKAMOTO) ---
Fuzzifikasi Suhu (38.5): Demam=0.500, Tinggi=0.167
Alpha Rule 1: 0.167, Z1: 66.667
Alpha Rule 2: 0.500, Z2: 80.000
Diagnosa Risiko DBD: 76.6667


--- HASIL BAGIAN B: PREDIKSI JAGUNG (ANFIS SUGENO) ---
Fuzzifikasi Moisture (60): Low=0.0, High=0.667
Fuzzifikasi Temp (26): Low=0, High=0.667
Weights: w1=0.0, w2=0.0, w3=0.0, w4=0.4444
Output Rule 4 (y4): 3.85795
Prediksi Produksi Jagung: 3.8580 ton/ha
